## Find Asset and Create Attachment

In [1]:
# Find existing Asset and create an Attachment
#
# Main function, establishes a connection to RKVST using an App Registration then uses that
# to find existing Asset and create an attachment.
#
# The attachment is an image located in a folder named "test_files"
#
# Note: The purpose of RKVST Jupyter Notebooks is to provide simplified examples that one can easily execute and digest.
# The RKVST Python SDK is authored to work cleanly with more advanced coding techniques.
#
# RKVST Python SDK: https://github.com/jitsuin-inc/archivist-python
#

In [2]:
from json import dumps as json_dumps
from os import getenv
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.proof_mechanism import ProofMechanism
from archivist.logger import set_logger

In [3]:
# URL, CLIENT, SECRET are environment variables that represent connection parameters.
#
# URL = represents the url to the RKVST application
# CLIENT = represents the client ID from an Application Registration
# SECRET = represents the client secret from an Application Registration
RKVST_URL = getenv("RKVST_URL")
APPREG_CLIENT = getenv("RKVST_APPREG_CLIENT")
APPREG_SECRET = getenv("RKVST_APPREG_SECRET")
RKVST_UNIQUE_ID = getenv("RKVST_UNIQUE_ID")
if not RKVST_UNIQUE_ID:
    raise Exception("RKVST_UNIQUE_ID is undefined")
RKVST_ARTIST_ATTACHMENT = getenv("RKVST_ARTIST_ATTACHMENT")
if not RKVST_ARTIST_ATTACHMENT:
    raise Exception("RKVST_ARTIST_ATTACHMENT is undefined")

In [4]:
"""
Main function of Asset and Event creation.

* Connect to RKVST with client ID and client secret
* Creates an Asset and two Events
* Prints response of Asset and Event creation
"""

# Optional call to set the logger level.  The argument can be either
# "INFO" or "DEBUG".  For more sophisticated logging control see our
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
print("URL", RKVST_URL)
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET), max_time=300)

Connecting to RKVST
URL https://app.dev-paul-0.wild.jitsuin.io


In [5]:
def upload_attachment(arch, path, name):
    """
    Obtains Attachments from "test_files" folder and creates json payload
    for upload
    """
    with open(RKVST_ARTIST_ATTACHMENT, "rb") as fd:
        blob = arch.attachments.upload(fd)
        attachment = {
            "arc_display_name": name,
            "arc_attachment_identity": blob["identity"],
            "arc_hash_value": blob["hash"]["value"],
            "arc_hash_alg": blob["hash"]["alg"],
        }
        return attachment

In [6]:
def get_artist(arch, name, artistid):
    """
    Finds existing Artist asset by name and unique id
    """
    attrs = {
        "arc_display_type": "Artists",
        "arc_display_name": name,
        "artistid": artistid,
    }

    return arch.assets.read_by_signature(attrs=attrs)

In [7]:
def create_image(arch, asset):
    """
    Creates an Event that uploads a primary image for an Artist asset
    """
    attachments = upload_attachment(
        arch, "pexels-andrea-turner-707697.jpeg", "arc_primary_image"
    )
    attachment_list = [attachments]

    props = {"operation": "Record", "behaviour": "RecordEvidence"}
    attrs = {
        "arc_description": "Attaching an image",
        "arc_display_type": "Primary image",
    }

    asset_attrs = {"arc_attachments": attachment_list}

    return arch.events.create(
        asset["identity"], props=props, attrs=attrs, asset_attrs=asset_attrs
    )

In [8]:
# Finding existing artist by name and artist id
# Fill in the artistid from Create Asset and Events notebook
print("Finding Asset")
asset = get_artist(arch, "Adele Laurie Blue Adkins", RKVST_UNIQUE_ID)
print("Asset", json_dumps(asset, indent=4))

Finding Asset


Refresh token


Asset {
    "identity": "assets/95de60d1-040b-4aff-a0f8-f042334d6ca6",
    "behaviours": [
        "AssetCreator",
        "Attachments",
        "RecordEvidence",
        "Builtin"
    ],
    "attributes": {
        "genre": "Soul",
        "stage_name": "Adele",
        "arc_description": "British Soul Singer",
        "arc_display_name": "Adele Laurie Blue Adkins",
        "arc_display_type": "Artists",
        "artistid": "3242643639"
    },
    "confirmation_status": "CONFIRMED",
    "tracked": "TRACKED",
    "owner": "0x3E06AcBf002E8F67bDe832b3f3E7d2aDA2BE3DC8",
    "at_time": "2023-01-10T16:26:42Z",
    "storage_integrity": "TENANT_STORAGE",
    "proof_mechanism": "SIMPLE_HASH",
    "chain_id": "99",
    "public": false,
    "tenant_identity": "tenant/f9040e8b-2afa-48f1-bb00-f055d55ade01"
}


In [9]:
# Create an Event for Attachment upload
print("Creating Events for existing Asset")
image_event = create_image(arch, asset)
print("Event for Image", json_dumps(image_event, indent=4))

Creating Events for existing Asset
Event for Image {
    "identity": "assets/95de60d1-040b-4aff-a0f8-f042334d6ca6/events/dfbfea31-e97d-4f88-abc8-169eeaa659a8",
    "asset_identity": "assets/95de60d1-040b-4aff-a0f8-f042334d6ca6",
    "event_attributes": {
        "arc_display_type": "Primary image",
        "arc_description": "Attaching an image"
    },
    "asset_attributes": {},
    "operation": "Record",
    "behaviour": "RecordEvidence",
    "timestamp_declared": "2023-01-10T16:26:48Z",
    "timestamp_accepted": "2023-01-10T16:26:48Z",
    "timestamp_committed": "2023-01-10T16:26:48.094799420Z",
    "principal_declared": {
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io/appidpv1",
        "subject": "bd3f85d2-9b92-4ef0-82c9-2bcb7b5d25c6",
        "display_name": "Application display name 05f19055-d2fd-4076-bd02-46a924cbe984",
        "email": ""
    },
    "principal_accepted": {
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io/appidpv1",
        "subject": "bd3f8